# Préparer un voyage avec Kayak

## Récupération des données météo
Pour récupérer les données météo, on utilisera l'API d'Open Weather (https://openweathermap.org/api).

Or celle-ci fonctionne avec des coordonnées GPS, il faut donc récupérer pour chaque ville sa latitude et sa longitude.

On utilisera alors l'API https://nominatim.org/release-docs/develop/api/Search/ qui permet de récupérer des coordonnés en fonction d'une recherche par addresse.

In [22]:
import pandas as pd
import numpy as np
import requests

try:
    df = pd.read_csv("data/df.csv", sep=';', index_col=0)
except:
    df = pd.read_json("data/cities.json")
    df = df.rename(columns={0 : "City"})
    df["Latitude"] = np.nan
    df["Longitude"] = np.nan
    df["Prain"] = np.nan
    df["Vrain"] = np.nan
    df["Temp"] = np.nan


In [2]:
def get_coordinates(city):
    """Return tuple(lat, long)"""

    display(f"get_coordinates {city}")
    r = requests.get("https://nominatim.openstreetmap.org/search", params= {"city": city, "country": "FRANCE", "format": "jsonv2"})
    if (r.status_code == 200):
        json = r.json()[0]
        return (json["lat"], json["lon"])
    else:
        raise ValueError("API did not return 200")

In [19]:
for index, row in df.iterrows():
    if (pd.isna(row["Latitude"]) == False and pd.isna(row["Longitude"]) == False):
        continue

    try:
        coordinates = get_coordinates(row["City"])
        display(coordinates)
        df.loc[df['City'] == row["City"], ['Latitude']] = coordinates[0]
        df.loc[df['City'] == row["City"], ['Longitude']] = coordinates[1]
    except ValueError as e:
       display(f"{row['City']} : {e}")


df

,City,Latitude,Longitude,Prain,Vrain,Temp
0,Mont Saint Michel,48.635954,-1.511460,NaN,NaN,NaN
1,St Malo,48.649518,-2.026041,NaN,NaN,NaN
2,Bayeux,49.276462,-0.702474,NaN,NaN,NaN
3,Le Havre,49.493898,0.107973,NaN,NaN,NaN
4,Rouen,49.440459,1.093966,NaN,NaN,NaN
5,Paris,48.858890,2.320041,NaN,NaN,NaN
6,Amiens,49.894171,2.295695,NaN,NaN,NaN
7,Lille,50.636565,3.063528,NaN,NaN,NaN
8,Strasbourg,48.584614,7.750713,NaN,NaN,NaN
9,Chateau du Haut Koenigsbourg,48.249523,7.345492,NaN,NaN,NaN


Maintenant que l'on a recuéperer les coordonnées GPS de chaque ville, on peut récupérer les infomations météorologique de ces villes dans les 7 prochains jours.

In [8]:
weather_api_key = 'd18cb64895afdf1a4079d65e40103694'
weather_domain = 'https://api.openweathermap.org'

In [33]:
def get_weather(city, lat, long):
    """Return tuple(rain probabilty, rain volume in mm, day temperature)"""

    print(f"get_weather {city}")
    r = requests.get(f"{weather_domain}/data/2.5/forecast", params= {"lat": lat, "lon": long, "units": "metric", "appid": weather_api_key, "mode": "json"})
    display(r.content)
    if (r.status_code == 200):
        json = r.json()["list"][len(r.json()["list"]) - 1]
        pop = json["pop"]
        rain = json["rain"]["3h"] if "rain" in json else 0
        temp = json["main"]["temp"]
        return (pop, rain, temp)
    else:
        raise ValueError("API did not return 200")

In [34]:
for index, row in df.iterrows():
    if (pd.isna(row["Prain"]) == False and pd.isna(row["Vrain"]) == False and pd.isna(row["Temp"]) == False):
        continue

    try:
        weather = get_weather(row["City"], row["Latitude"], row["Longitude"])
        display(weather)
        df.loc[df['City'] == row["City"], ['Prain']] = weather[0]
        df.loc[df['City'] == row["City"], ['Vrain']] = weather[1]
        df.loc[df['City'] == row["City"], ['Temp']] = weather[2]
    except ValueError as e:
       display(f"{row['City']} : {e}")


df

get_weather Paris


b'{"cod":"200","message":0,"cnt":40,"list":[{"dt":1695913200,"main":{"temp":23.1,"feels_like":22.75,"temp_min":23.1,"temp_max":24.09,"pressure":1017,"sea_level":1017,"grnd_level":1012,"humidity":49,"temp_kf":-0.99},"weather":[{"id":800,"main":"Clear","description":"clear sky","icon":"01d"}],"clouds":{"all":0},"wind":{"speed":3.76,"deg":224,"gust":6.11},"visibility":10000,"pop":0,"sys":{"pod":"d"},"dt_txt":"2023-09-28 15:00:00"},{"dt":1695924000,"main":{"temp":22.88,"feels_like":22.58,"temp_min":22.45,"temp_max":22.88,"pressure":1017,"sea_level":1017,"grnd_level":1012,"humidity":52,"temp_kf":0.43},"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03n"}],"clouds":{"all":30},"wind":{"speed":1.9,"deg":201,"gust":4.17},"visibility":10000,"pop":0,"sys":{"pod":"n"},"dt_txt":"2023-09-28 18:00:00"},{"dt":1695934800,"main":{"temp":21.75,"feels_like":21.63,"temp_min":21.08,"temp_max":21.75,"pressure":1017,"sea_level":1017,"grnd_level":1012,"humidity":63,"temp_kf":0.67}

(0.06, 0, 20.18)

get_weather Amiens


b'{"cod":"200","message":0,"cnt":40,"list":[{"dt":1695913200,"main":{"temp":20.22,"feels_like":19.52,"temp_min":20.22,"temp_max":21.48,"pressure":1016,"sea_level":1016,"grnd_level":1013,"humidity":47,"temp_kf":-1.26},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04d"}],"clouds":{"all":100},"wind":{"speed":4.31,"deg":222,"gust":7.48},"visibility":10000,"pop":0,"sys":{"pod":"d"},"dt_txt":"2023-09-28 15:00:00"},{"dt":1695924000,"main":{"temp":19.45,"feels_like":18.78,"temp_min":17.9,"temp_max":19.45,"pressure":1016,"sea_level":1016,"grnd_level":1013,"humidity":51,"temp_kf":1.55},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":100},"wind":{"speed":3.07,"deg":183,"gust":4.28},"visibility":10000,"pop":0,"sys":{"pod":"n"},"dt_txt":"2023-09-28 18:00:00"},{"dt":1695934800,"main":{"temp":18.94,"feels_like":18.48,"temp_min":18.3,"temp_max":18.94,"pressure":1016,"sea_level":1016,"grnd_level":1012,"humidity":61,"temp

...

(0.24, 0.54, 20.64)

get_weather La Rochelle


b'{"cod":"200","message":0,"cnt":40,"list":[{"dt":1695913200,"main":{"temp":25.07,"feels_like":25.33,"temp_min":22.25,"temp_max":25.07,"pressure":1016,"sea_level":1016,"grnd_level":1016,"humidity":65,"temp_kf":2.82},"weather":[{"id":800,"main":"Clear","description":"clear sky","icon":"01d"}],"clouds":{"all":0},"wind":{"speed":3.7,"deg":255,"gust":4.24},"visibility":10000,"pop":0,"sys":{"pod":"d"},"dt_txt":"2023-09-28 15:00:00"},{"dt":1695924000,"main":{"temp":23.59,"feels_like":23.86,"temp_min":20.63,"temp_max":23.59,"pressure":1016,"sea_level":1016,"grnd_level":1017,"humidity":71,"temp_kf":2.96},"weather":[{"id":800,"main":"Clear","description":"clear sky","icon":"01n"}],"clouds":{"all":3},"wind":{"speed":3.47,"deg":251,"gust":4.78},"visibility":10000,"pop":0,"sys":{"pod":"n"},"dt_txt":"2023-09-28 18:00:00"},{"dt":1695934800,"main":{"temp":21.92,"feels_like":22.2,"temp_min":20.34,"temp_max":21.92,"pressure":1017,"sea_level":1017,"grnd_level":1018,"humidity":78,"temp_kf":1.58},"weather

(0.05, 0, 19.37)

,City,Latitude,Longitude,Prain,Vrain,Temp
0,Mont Saint Michel,48.635954,-1.511460,0.00,0.00,18.03
1,St Malo,48.649518,-2.026041,0.00,0.00,17.24
2,Bayeux,49.276462,-0.702474,0.00,0.00,18.97
3,Le Havre,49.493898,0.107973,0.00,0.00,16.98
4,Rouen,49.440459,1.093966,0.00,0.00,18.82
5,Paris,48.858890,2.320041,0.06,0.00,20.18
6,Amiens,49.894171,2.295695,0.00,0.00,18.41
7,Lille,50.636565,3.063528,0.33,0.00,18.78
8,Strasbourg,48.584614,7.750713,0.33,0.33,22.47
9,Chateau du Haut Koenigsbourg,48.249523,7.345492,0.30,0.60,19.94


In [21]:
# save to csv to not recall API when the kernel restart
df.to_csv("data/df.csv", sep=';')